This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/pick.html). I recommend having both windows open, side-by-side!

# UR5e Touch-Hold-Retreat Motion

Adapted from the IIWA pick-and-place example for the UR5e with NetFT probe.

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
from pydrake.all import (
    DiagramBuilder,
    Integrator,
    JacobianWrtVariable,
    LeafSystem,
    Multiplexer,
    PiecewisePose,
    Rgba,
    RigidTransform,
    RotationMatrix,
    Simulator,
    StartMeshcat,
    TrajectorySource,
)
from manipulation import running_as_notebook
from manipulation.station import LoadScenario, MakeHardwareStation
from ur5e_description.ur5e_driver import UR5E_PACKAGE_XML

In [ ]:
meshcat = StartMeshcat()

# TCP Contact Frames

For touch-hold-retreat, we define keyframe poses for the TCP. The TCP approaches from above with Z-axis pointing down.

In [ ]:
def MakeTCPFrames(X_TCP, X_BrickA, X_BrickB):
    """Creates keyframe poses for touch-hold-retreat motion."""
    R_touch = RotationMatrix.MakeXRotation(np.pi)  # Z pointing down
    brick_half_height, approach_height, contact_offset = 0.025, 0.08, 0.005
    
    # Brick A
    p_A = X_BrickA.translation()
    top_A = p_A[2] + brick_half_height
    X_TCP["pretouch_A"] = RigidTransform(R_touch, [p_A[0], p_A[1], top_A + approach_height])
    X_TCP["touch_A"] = RigidTransform(R_touch, [p_A[0], p_A[1], top_A - contact_offset])
    X_TCP["posttouch_A"] = X_TCP["pretouch_A"]
    
    # Clearance
    p_mid = (p_A + X_BrickB.translation()) / 2
    X_TCP["clearance"] = RigidTransform(R_touch, [p_mid[0], p_mid[1], top_A + 0.15])
    
    # Brick B
    p_B = X_BrickB.translation()
    top_B = p_B[2] + brick_half_height
    X_TCP["pretouch_B"] = RigidTransform(R_touch, [p_B[0], p_B[1], top_B + approach_height])
    X_TCP["touch_B"] = RigidTransform(R_touch, [p_B[0], p_B[1], top_B - contact_offset])
    X_TCP["posttouch_B"] = X_TCP["pretouch_B"]
    X_TCP["final"] = X_TCP["initial"]
    
    # Timing
    speed_factor = 10.0
    times = {"initial": 0.0}
    def add_time(prev, curr, dur=None):
        if dur is None:
            dur = speed_factor * np.linalg.norm(X_TCP[curr].translation() - X_TCP[prev].translation())
        times[curr] = times[prev] + max(dur, 0.5)
    
    add_time("initial", "pretouch_A")
    times["touch_A_start"] = times["pretouch_A"] + 1.0
    times["touch_A_end"] = times["touch_A_start"] + 1.0
    X_TCP["touch_A_start"] = X_TCP["touch_A_end"] = X_TCP["touch_A"]
    times["posttouch_A"] = times["touch_A_end"] + 1.0
    add_time("posttouch_A", "clearance")
    add_time("clearance", "pretouch_B")
    times["touch_B_start"] = times["pretouch_B"] + 1.0
    times["touch_B_end"] = times["touch_B_start"] + 1.0
    X_TCP["touch_B_start"] = X_TCP["touch_B_end"] = X_TCP["touch_B"]
    times["posttouch_B"] = times["touch_B_end"] + 1.0
    add_time("posttouch_B", "final")
    
    return X_TCP, times

def MakeTCPPoseTrajectory(X_TCP, times):
    """Constructs PiecewisePose trajectory from keyframes."""
    order = ["initial", "pretouch_A", "touch_A_start", "touch_A_end", "posttouch_A",
             "clearance", "pretouch_B", "touch_B_start", "touch_B_end", "posttouch_B", "final"]
    return PiecewisePose.MakeLinear([times[k] for k in order], [X_TCP[k] for k in order])

# Pseudo-Inverse Controller

In [ ]:
class PseudoInverseController(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._ur5e = plant.GetModelInstanceByName("ur5e")
        self._TCP = plant.GetBodyByName("tcp", plant.GetModelInstanceByName("netft_probe")).body_frame()
        self._W = plant.world_frame()

        self.DeclareVectorInputPort("V_W_TCP", 6)
        self.DeclareVectorInputPort("ur5e.state", 12)
        self.DeclareVectorOutputPort("ur5e.velocity", 6, self.CalcOutput)

    def CalcOutput(self, context, output):
        V_TCP = self.get_input_port(0).Eval(context)
        q = self.get_input_port(1).Eval(context)[:6]
        self._plant.SetPositions(self._plant_context, self._ur5e, q)
        J = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kQDot, self._TCP, [0,0,0], self._W, self._W)
        output.SetFromVector(np.linalg.pinv(J) @ V_TCP)

# Build and Run Simulation

In [ ]:
# Define brick positions
X_BrickA = RigidTransform(RotationMatrix.MakeZRotation(0), [0.4, -0.2, 0.0])
X_BrickB = RigidTransform(RotationMatrix.MakeZRotation(0), [0.4, 0.2, 0.0])

# Scenario YAML - same pattern as IIWA example
scenario_data = f"""
directives:
- add_directives:
    file: package://ur5e_description/ur5e_netft_probe.dmd.yaml
- add_model:
    name: brick_a
    file: package://manipulation/hydro/061_foam_brick.sdf
- add_weld:
    parent: world
    child: brick_a::base_link
    X_PC:
      translation: [{X_BrickA.translation()[0]}, {X_BrickA.translation()[1]}, {X_BrickA.translation()[2]}]
- add_model:
    name: brick_b
    file: package://manipulation/hydro/061_foam_brick.sdf
- add_weld:
    parent: world
    child: brick_b::base_link
    X_PC:
      translation: [{X_BrickB.translation()[0]}, {X_BrickB.translation()[1]}, {X_BrickB.translation()[2]}]
model_drivers:
    ur5e: !InverseDynamicsDriver {{}}
"""

meshcat.Delete()
builder = DiagramBuilder()
scenario = LoadScenario(data=scenario_data)
station = builder.AddSystem(MakeHardwareStation(
    scenario, meshcat=meshcat, package_xmls=[UR5E_PACKAGE_XML]))
station.set_name("station")
plant = station.GetSubsystemByName("plant")

# Get initial TCP pose
temp_context = station.CreateDefaultContext()
tcp_body = plant.GetBodyByName("tcp", plant.GetModelInstanceByName("netft_probe"))
X_TCP_initial = plant.EvalBodyPoseInWorld(plant.GetMyContextFromRoot(temp_context), tcp_body)
print(f"Initial TCP position: {X_TCP_initial.translation()}")

# Generate trajectory
X_TCP, times = MakeTCPFrames({"initial": X_TCP_initial}, X_BrickA, X_BrickB)
traj = MakeTCPPoseTrajectory(X_TCP, times)
traj_V = traj.MakeDerivative()
print(f"Total duration: {times['final']:.1f}s")

# Visualize path
traj_p = traj.get_position_trajectory()
meshcat.SetLine("tcp_path", traj_p.vector_values(traj_p.get_segment_times()), 2.0, rgba=Rgba(1, 0.65, 0))

In [ ]:
# Wire control diagram
V_source = builder.AddSystem(TrajectorySource(traj_V))
controller = builder.AddSystem(PseudoInverseController(plant))
integrator = builder.AddSystem(Integrator(6))
mux = builder.AddSystem(Multiplexer([6, 6]))

builder.Connect(V_source.get_output_port(), controller.get_input_port(0))
builder.Connect(station.GetOutputPort("ur5e.state_estimated"), controller.get_input_port(1))
builder.Connect(controller.get_output_port(), integrator.get_input_port())
builder.Connect(integrator.get_output_port(), mux.get_input_port(0))
builder.Connect(controller.get_output_port(), mux.get_input_port(1))
builder.Connect(mux.get_output_port(), station.GetInputPort("ur5e.desired_state"))

diagram = builder.Build()
diagram.set_name("ur5e_touch_motion")

In [ ]:
# Run simulation
simulator = Simulator(diagram)
context = simulator.get_mutable_context()

# Initialize integrator
q0 = plant.GetPositions(plant.GetMyContextFromRoot(context), plant.GetModelInstanceByName("ur5e"))
integrator.set_integral_value(integrator.GetMyContextFromRoot(context), q0)

diagram.ForcedPublish(context)
meshcat.StartRecording(set_visualizations_while_recording=False)
simulator.AdvanceTo(times["final"] if running_as_notebook else 0.1)
meshcat.PublishRecording()
print("Done! Check meshcat.")

# Hardware Mode

For real UR5e hardware, replace `MakeHardwareStation` with `UR5eDriver`:

```python
from ur5e_description.ur5e_driver import UR5eDriver

# Instead of MakeHardwareStation, use:
driver = builder.AddSystem(UR5eDriver(robot_ip="192.168.1.102"))

# Same port names - connections unchanged:
# builder.Connect(..., driver.GetInputPort("ur5e.desired_state"))
# builder.Connect(driver.GetOutputPort("ur5e.state_estimated"), ...)
```